In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
GPS    = np.genfromtxt('rates.csv', delimiter=',', names=True, case_sensitive=True) 
faults = np.genfromtxt('faults.csv', delimiter=',', names=True, case_sensitive=True) 
coasts = np.genfromtxt('coasts.csv', delimiter=',', names=True, case_sensitive=True)


the original "map" locations have values like 4e6 meters - this just makes
the plots a little cleaner.  Doesn't affect any of the math at all.

In [ ]:
coasts['x']=coasts['x']-GPS['x'].mean()
coasts['y']=coasts['y']-GPS['y'].mean()
faults['x']=faults['x']-GPS['x'].mean()
faults['y']=faults['y']-GPS['y'].mean()
GPS['x']=GPS['x']-GPS['x'].mean()
GPS['y']=GPS['y']-GPS['y'].mean()

GPS['UE']=GPS['UE']/1000 #%convert to meters/yr, since our units of distance are in meters
GPS['UN']=GPS['UN']/1000

print(GPS['UN'].mean())

All the stuff above is old, the stuff below is new. Walk through the code step by step, there are some variables and equations that you need to change.

First, plot velocity vectors on normal plot (E,W axes). No changes necessary here, but read notes.

Note that when we remove the mean of the GPS vectores, it doesn't necessarily mean that all of the vectors on one side of the fault goes one way and the ones on the other side go the other way.  If we have more points on one side, then you'll get a mean value weighted towards those values.  But it makes it a little easier to see the complexity.

In [ ]:
fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.plot(coasts['x']/1e3,coasts['y']/1e3,'b')
ax.plot(faults['x']/1e3,faults['y']/1e3,'r',linewidth=0.5)

#add displacement rate vectors, as well as scale arrow 
ax.quiver(GPS['x']/1e3,GPS['y']/1e3, GPS['UE'],GPS['UN'],color='k')

plt.axis('equal')
plt.xlim([-3.5e2, 3e2])
plt.ylim([-2.5e2, 2e2])
plt.xlabel('Easting (km)')
plt.ylabel('Northing (km)')
plt.title('Data')
plt.show()

### Rotate region
We want to plot the rates along a profile perpendicular to the fault.  One way to do this is to rotate the whole region, and then just pull out a given range of "Y" values in the new coordinate system.  Try various values of "theta", for the angle of rotation, until the San Andreas is essentially up-down on your plot (i.e., vectors on one side point up, on the other side they point down).

In [ ]:
###CHANGE THIS
theta    = 10  #in degrees
thetarad = np.radians(theta)

#This is a 2x2 matrix to rotate the coordinates of coastlines, faults, etc.
rotM = np.array([[np.cos(thetarad),np.sin(thetarad)],[-np.sin(thetarad),np.cos(thetarad)]])

#Now rotate all the things we've been plotting
fault_coords = np.column_stack((faults['x'],faults['y'])).transpose()
rot_fault    = rotM.dot(fault_coords)
rotfx        = rot_fault[0,:]
rotfy        = rot_fault[1,:]

coast_coords = np.column_stack((coasts['x'],coasts['y'])).transpose()
rot_coast    = rotM.dot(coast_coords)
rotcx        = rot_coast[0,:]
rotcy        = rot_coast[1,:]

GPS_coords   = np.column_stack((GPS['x'],GPS['y'])).transpose()
rot_GPS      = rotM.dot(GPS_coords)
rotGPS_x     = rot_GPS[0,:]
rotGPS_y     = rot_GPS[1,:]

GPS_displ    = np.column_stack((GPS['UE'],GPS['UN'])).transpose()
rot_GPSd     = rotM.dot(GPS_displ)
rotGPS_UE    = rot_GPSd[0,:]
rotGPS_UN    = rot_GPSd[1,:]


In [ ]:
#Plot the variables in the rotated coordinate system
fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)

#add displacement rate vectors
ax.quiver(rotGPS_x,rotGPS_y, rotGPS_UE-rotGPS_UE.mean(), rotGPS_UN-rotGPS_UN.mean(),color='k',scale=2e-1)

plt.axis('image')
ax.plot(rotcx,rotcy,'b')
ax.plot(rotfx,rotfy,'r',linewidth=0.5)

plt.xlim([rotGPS_x.min(),rotGPS_x.max()])
plt.ylim([rotGPS_y.min(),rotGPS_y.max()])
plt.xlabel('rotated x coords (m)')
plt.ylabel('rotated y coords (m)')
plt.title('Data, rotated')
plt.show()


### Select range of Y-values and plot profile
The San Andreas does not run perfectly straigh - it has a stepover north of Los Angeles.  In your rotated plot, you should be able to see that there is a horizontal offset between the upper and lower sections.  If you plot the whole thing vs your new, rotated X coordinate, you will get the following plot.


In [ ]:
fig=plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(rotGPS_x,rotGPS_UN,'r.')
plt.xlabel('Distance across fault, rotated coords, meters')
plt.ylabel('Fault-parallel displacement rate (m/yr)')
plt.title('Data along profile (rotated x coords)')
plt.grid(True)
plt.show()


The deformation zone looks really broad and smeared out because you are combining the northern and southern sections together.  Instead, let's pick JUST the southern section.  To do that, we will use ONLY values that fall within certain bounds in the rotated Y variable. Use the Y axis as shown in your previous, rotated map view plot.

In [ ]:
###CHANGE THESE to actual numbers, they are set to a default max/min in your rotated coords.
min_roty = rotGPS_y.min()
max_roty = rotGPS_y.max()

goodid = np.where((rotGPS_y < max_roty) & (rotGPS_y > min_roty))

fig=plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(rotGPS_x[goodid]/1e3,rotGPS_UN[goodid],'r.')
plt.xlabel('Distance across fault, rotated coords, meters')
plt.ylabel('Fault-parallel displacement rate (m/yr)')
plt.title('Data along profile (rotated x coords)')
plt.grid(True)
plt.show()


### Model interseismic
Now you need to model the fault, using the equation shown from class. Tinker with the following:

In [ ]:
# Location of fault (this should be about where the center of your
# "arctan" signal is, in meters (note that the last figure shows km)
x0 = 5e3

# Locking depth, in meters
D = 35e3

# Plate rate (in meters/ year, like your UE, UN vectors.  Should be about
# the total from one side of the fault to the other. The negative sign
# makes the value right-lateral (left side positive)
V = -0.08;

### Predicted profile of displacement rate, at just our selected points.  
### This equation is WRONG!  Fix it to what we showed in class.

In [ ]:
model = V/np.pi*np.arccos(D/(rotGPS_x[goodid]-x0))

## Plot model and data

In [ ]:
#solve for the average offset of this curve (i.e., center on the data, up or down)
residual = rotGPS_UN[goodid]-model
meanrate = residual.mean()
rms      = residual.std()


fig=plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot((rotGPS_x[goodid]-x0)/1e3,rotGPS_UN[goodid]-meanrate,'.', label='Data')
ax.plot((rotGPS_x[goodid]-x0)/1e3,model,'r.', label='Model')
ax.legend()
plt.xlabel('Distance across fault, rotated coords, meters')
plt.ylabel('Fault-parallel displacement rate (m/yr)')
plt.title('Data along profile (rotated x coords)')
plt.grid(True)
plt.show()


#    title(['std dev of res. = ' num2str(rms*100) ' cm/yr'])



In [ ]:
dx = 30e3 #grid spacing in meters
gridx = np.arange(GPS['x'].min(),GPS['x'].max(),dx) #this makes a vector of x values, spaced by dx
gridy = np.arange(GPS['y'].min(),GPS['y'].max(),dx) #this makes a vector of y values, spaced by dx

#how many x and y points do we have?  We use that later.
nx=np.size(gridx); 
ny=np.size(gridy);

#This makes a grid of x and y values from our two vectors
[x,y]=np.meshgrid(gridx,gridy)

#smoothing distance - this allows us to use points that are not just within
#our single grid box, while still making the ones in the box more
#"important".  Change this up!
alpha = 10e3; #in meters


In [ ]:
m     = np.zeros([6,ny,nx]); #initialize an empty model vector 
count = x*0;                 #keep a vector of how many points we end up using for each box


In [ ]:
for i in np.arange(0,ny): #loop over all ny rows
    for j in np.arange(0,nx): #loop over all nx columns
        #Calculate the distance from ALL GPS points to your grid box center.
        dists=np.sqrt(np.square(GPS['x']-x[i,j])+np.square(GPS['y']-y[i,j]))

        #this is the equation for a Gaussian curve, with width alpha
        weights = np.exp(-(np.square(dists))/(2*np.square(alpha)))       
        
        #only use points that are within 2*alpha from the center of our box, or the box size itself
        ind     = np.where((dists < 2*alpha) | (dists < dx/2))
        nd      = np.size(ind);   # number of "good" points
        
        sig      =  np.diag(np.hstack((weights[ind],weights[ind])))

        #data vector of just the points we are using
        d        = np.hstack((GPS['UE'][ind],GPS['UN'][ind]))
        z        = np.zeros(nd)
        o        = np.ones(nd)

        #%build G matrix
        G1    = np.column_stack((o,z,GPS['x'][ind],GPS['y'][ind],z,z))  #first set, UE 
        G2    = np.column_stack((z,o,z,z,GPS['x'][ind],GPS['y'][ind])) #2nd set, UN
        G     = np.concatenate((G1,G2),axis=0)
    
        Gt   = np.transpose(G)
        G3   = (Gt.dot(sig)).dot(G);
        
        # If the next value is very big or infinite, 
        #this means the matrix is not invertible, 
        #probably because you have < 6 points or 
        #points that are very close together
 
        rcond = np.linalg.cond(G3) 
    
        if np.isinf(rcond) or rcond >1e16:
            m[:,i,j]=np.nan
           
        else:
            Gg       = (np.linalg.inv(G3).dot(Gt)).dot(sig)
            m[:,i,j] = Gg.dot(d)
            count[i,j]=nd


Now we want to pull out the extension and shear components

In [ ]:
dilatation = x*np.nan
shear      = x*np.nan

for i in np.arange(0,ny): #loop over all ny rows
    for j in np.arange(0,nx): #loop over all nx columns
        L  = [[m[2,i,j],m[3,i,j]],[m[4,i,j],m[5,i,j]]]
        W = (L-np.transpose(L))/2
        E = (L+np.transpose(L))/2
          
        dilatation[i,j] = np.trace(E)
        shear[i,j]      = E[0,0]-E[1,1]


In [ ]:
fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)    
im=ax.pcolor(x,y,dilatation,vmin=-1e-7,vmax=1e-7)
ax.plot(faults['x'],faults['y'],'r')
ax.plot(GPS['x'],GPS['y'],'k.')
plt.axis('image')
plt.xlabel('East (m)')
plt.ylabel('North (m)')
plt.xlim(GPS['x'].min(), GPS['x'].max())
plt.ylim(GPS['y'].min(), GPS['y'].max())
plt.title('dilatation')
fig.colorbar(im,ax=ax)
plt.show()

fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)     
im=ax.pcolor(x,y,shear,vmin=0,vmax=2e-7)
ax.plot(faults['x'],faults['y'],'r')
ax.plot(GPS['x'],GPS['y'],'k.')
plt.axis('image')
plt.xlabel('East (m)')
plt.ylabel('North (m)')
plt.xlim(GPS['x'].min(), GPS['x'].max())
plt.ylim(GPS['y'].min(), GPS['y'].max())
plt.title('shear');
fig.colorbar(im,ax=ax)
plt.show()
